PARTE 1: RED NEURONAL


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import os

# uploading el dataset
if not os.path.exists('/content/drive/MyDrive/Proyecto_Final_SIC/dataset_listo_para_entrenar.csv'):
    print("Error: dataset no enceontrado")
    exit()

df = pd.read_csv('/content/drive/MyDrive/Proyecto_Final_SIC/dataset_listo_para_entrenar.csv')

# Definimos Entrada (X) y Salida (y)
# X = El precio limpio
# y = La categoría (0, 1, 2)
X = df[['Total_Limpio']].values
y = df['Categoria'].values

# Preprocesamiento: hay qyue jugar con numeros grandes
# Las redes neuronales funcionan mal con números grandes (como 1000, 5000).
# El Scaler convierte esos números a una escala pequeña (ej: 0.5, -1.2) para que la red aprenda rápido.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# aplico data spliting: 80% para entrenar, 20% para examinar qué tan bien aprendió
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#Arquitectura de la red neuronal
model = tf.keras.models.Sequential([
    # Capa de entrada y oculta 1: 64 neuronas
    tf.keras.layers.Dense(64, activation='relu', input_shape=(1,)),

    # Capa oculta 2: 32 neuronas
    tf.keras.layers.Dense(32, activation='relu'),

    # Capa de SALIDA: 3 neuronas (porque tenemos 3 categorías: Bajo, Medio, Alto)
    # Softmax convierte los resultados en probabilidades
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compilacionn
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Usamos esta loss function porque las etiquetas son enteros (0,1,2)
              metrics=['accuracy'])

#training
print("--- INICIANDO ENTRENAMIENTO ---")
# Epochs = Cuántas veces repasará los datos (50 veces)
historia = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1, validation_data=(X_test, y_test))

#metricas
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nPRECISIÓN FINAL DEL MODELO: {accuracy * 100:.2f}%")

# 6. Guardamos los archivos necesarios pa la aplicación final
# Guardamos el modelo
model.save('modelo_facturas.keras')
# Guardamos el escalador
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\n todo bien. Se han creado 'modelo_facturas.keras' y 'scaler.pkl'.")

--- INICIANDO ENTRENAMIENTO ---
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6255 - loss: 0.8712 - val_accuracy: 0.7703 - val_loss: 0.4862
Epoch 2/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8319 - loss: 0.4232 - val_accuracy: 0.9611 - val_loss: 0.2503
Epoch 3/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9633 - loss: 0.2220 - val_accuracy: 0.9823 - val_loss: 0.1481
Epoch 4/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9768 - loss: 0.1383 - val_accuracy: 0.9894 - val_loss: 0.0997
Epoch 5/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9890 - loss: 0.1069 - val_accuracy: 0.9965 - val_loss: 0.0791
Epoch 6/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9858 - loss: 0.0839 - val_accuracy: 1.0000 - val_loss: 0.0675
Epoch 7/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9908 - loss: 0.0701 - val_accuracy: 0.9894 - val_loss: 0.0600
Epoch 8/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9878 - loss: 0.0623 - val_accuracy: 0.9965 - val_loss: 0.0544
Epo

PARTE 2: PROBANDO EL MODELO CON IMAGENES O PDFS

In [2]:
#teseracto y popler para trabajar con pdfs
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!sudo apt-get install poppler-utils
!pip install pytesseract pdf2image tensorflow scikit-learn pandas opencv-python-headless

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.5 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB

PARTE 3: CLASIFICANDO

In [3]:
import cv2
import pytesseract
import numpy as np
import tensorflow as tf
import pickle
import re
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import io

# Definimos las clases igual que en el entrenamiento
CLASES = {0: "BAJO", 1: "MEDIO", 2: "ALTO"}

# Cargo el  modelo y escalador
try:
    model = tf.keras.models.load_model('/content/drive/MyDrive/Proyecto_Final_SIC/modelo_facturas.keras')
    with open('/content/drive/MyDrive/Proyecto_Final_SIC/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    print("Modelo y Scaler cargados en memoria.")
except Exception as e:
    print(f"Error cargando el modelo: {e}")


# --- FUNCIONES DE LIMPIEZA Y OCR ---

def limpiar_precio_complejo(texto):
    """
    Busca el patrón 'Total $ XX,XX' y extrae el último valor numérico encontrado.
    """
    if not isinstance(texto, str): return 0.0
    # Regex ajustado para encontrar números con decimales
    patron = r'(\d+[\.,]\d+)'
    encontrados = re.findall(patron, texto)

    if encontrados:
        # Tomamos el último valor encontrado (por si hay subtotales antes)
        precio_final = encontrados[-1]
        # Reemplazamos coma por punto y quitamos espacios
        precio_final = precio_final.replace(',', '.').replace(' ', '')
        try:
            return float(precio_final)
        except ValueError:
            return 0.0
    return 0.0

def obtener_texto_de_archivo(nombre_archivo, contenido_bytes):
    """
    Detecta si es Imagen o PDF y extrae el texto usando OCR.
    """
    texto_completo = ""

    # CASO 1: Es un PDF
    if nombre_archivo.lower().endswith('.pdf'):
        print(" tipo de archivo: PDF. Convirtiendo a imagen para leer...")
        try:
            # Convertimos el PDF (bytes) a lista de imágenes
            imagenes = convert_from_path(nombre_archivo)

            # Procesamos solo la primera página
            texto_completo = pytesseract.image_to_string(imagenes[0])
            print("Lectura de PDF exitosa.")
        except Exception as e:
            print(f"Error leyendo PDF: {e}")

    # CASO 2: Es un .jpg o png
    else:
        print("imagen detectada. Leyendo...")
        try:
            # Convertir bytes a imagen PIL
            image = Image.open(io.BytesIO(contenido_bytes))
            texto_completo = pytesseract.image_to_string(image)
            print(" la imagen ha sido leida.")
        except Exception as e:
            print(f"Error leyendo Imagen: {e}")

    return texto_completo

# --- PROGRAMA PRINCIPAL ---

print("\n sube una factura pa analizar")
uploads = files.upload()

for nombre_archivo, contenido in uploads.items():
    print(f"\n--- Analizando: {nombre_archivo} ---")

    # se obtiene texto (OCR)
    texto_extraido = obtener_texto_de_archivo(nombre_archivo, contenido)

    # Buscamos especificamente la linea del Total
    # Usamos una búsqueda simple primero para ubicar la zona del 'Total'
    match = re.search(r'(Total.*)', texto_extraido, re.IGNORECASE | re.DOTALL)

    precio_detectado = 0.0

    if match:
        # Pasamos el fragmento de texto a nuestra función de limpieza
        linea_total = match.group(1)
        print(f"Texto relevante encontrado: '{linea_total[:50]}...'") # Mostramos un pedazo
        precio_detectado = limpiar_precio_complejo(linea_total)
    else:
        # Si no encuentra la palabra "Total", intenta buscar números al final del texto
        print("la palabra total no se ha encontrado. Intentando buscar números sueltos...")
        precio_detectado = limpiar_precio_complejo(texto_extraido)

    if precio_detectado > 0:
        print(f" PRECIO IDENTIFICADO: ${precio_detectado}")

        # Se preprocesa para la IA (Escalar)
        # La red espera una lista de listas [[valor]]
        precio_array = np.array([[precio_detectado]])
        precio_escalado = scaler.transform(precio_array)

        # fase de predicción
        prediccion = model.predict(precio_escalado, verbose=0)
        clase_id = np.argmax(prediccion) # El índice con mayor probabilidad
        confianza = np.max(prediccion) * 100

        resultado = CLASES[clase_id]

        print("\n" + "="*40)
        print(f"CLASIFICACIÓN IA: {resultado}")
        print(f"Confianza: {confianza:.2f}%")
        print("="*40)

    else:
        print("\n error: No se pudo identificar un precio válido en el documento.")
        print("Intenta subir una imagen con mejor calidad.")

Modelo y Scaler cargados en memoria.

 sube una factura pa analizar


Saving factura_rectificativa_2017_0001.png to factura_rectificativa_2017_0001.png

--- Analizando: factura_rectificativa_2017_0001.png ---
imagen detectada. Leyendo...
 la imagen ha sido leida.
Texto relevante encontrado: 'Total Base Imponible: 80,00 €

LV.A. 21%: -16,80 €...'
 PRECIO IDENTIFICADO: $84.8

CLASIFICACIÓN IA: BAJO
Confianza: 100.00%
